# Manipulating DataFrame (in-memory catalog)


In [ ]:
import warnings

warnings.filterwarnings("ignore")
import intake

The in-memory representation of an Earth System Model (ESM) catalog is a pandas
dataframe, and is accessible via the `.df` property:


In [ ]:
url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(url)
col.df.head()

In this notebook we will go through some examples showing how to manipulate this
dataframe outside of intake-esm.


## Use Case 1: Complex Search Queries

Let's say we are interested in datasets with the following attributes:

- `experiment_id=["historical"]`
- `table_id="Amon"`
- `variable_id="tas"`
- `source_id=['TaiESM1', 'AWI-CM-1-1-MR', 'AWI-ESM-1-1-LR', 'BCC-CSM2-MR', 'BCC-ESM1', 'CAMS-CSM1-0', 'CAS-ESM2-0', 'UKESM1-0-LL']`

In addition to these attributes, **we are interested in the first ensemble
member (member_id) of each model (source_id) only**.

This can be achieved in two steps:

### Step 1: Run a query against the catalog

We can run a query against the catalog:


In [ ]:
col_subset = col.search(
    experiment_id=["historical"],
    table_id="Amon",
    variable_id="tas",
    source_id=[
        "TaiESM1",
        "AWI-CM-1-1-MR",
        "AWI-ESM-1-1-LR",
        "BCC-CSM2-MR",
        "BCC-ESM1",
        "CAMS-CSM1-0",
        "CAS-ESM2-0",
        "UKESM1-0-LL",
    ],
)
col_subset

### Step 2: Select the first `member_id` for each `source_id`

The subsetted catalog contains `source_id` with the following number of
`member_id` per `source_id`:


In [ ]:
col_subset.df.groupby("source_id")["member_id"].nunique()

To get the first `member_id` for each `source_id`, we group the dataframe by
`source_id` and use the `.first()` function to retrieve the first `member_id`:


In [ ]:
grouped = col_subset.df.groupby(["source_id"])
df = grouped.first().reset_index()

# Confirm that we have one ensemble member per source_id

df.groupby("source_id")["member_id"].nunique()

### Step 3: Attach the new dataframe to our catalog object


In [ ]:
col_subset.df = df
col_subset

In [ ]:
dsets = col_subset.to_dataset_dict(zarr_kwargs={"consolidated": True})
[key for key in dsets]

In [ ]:
print(dsets["CMIP.CAS.CAS-ESM2-0.historical.Amon.gn"])

In [ ]:
import intake_esm  # just to display version information

intake_esm.show_versions()